In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sumolib
import sys
import json
import matplotlib.pyplot as plt
import numpy as np
import torch

sys.path.append('..')
from util_folder.ml_utils.data_utils.data_loader_utils import IncidentDataModule

In [3]:
# Load net and dataset info
folder_path = f'../Simulation_scenarios/motorway/Results/incident_large_long'
incident_data_module = IncidentDataModule(folder_path = folder_path, transform='scalenormalize', batch_size = 100)
incident_data_module.setup()
train_dataloader = incident_data_module.train_dataloader()
train_dataset = train_dataloader.dataset
val_dataloader = incident_data_module.val_dataloader()
val_dataset = val_dataloader.dataset
test_dataloader = incident_data_module.test_dataloader()
test_dataset = test_dataloader.dataset

net_path = '../Simulation_scenarios/motorway/Simulations/Base/network.net.xml'
net = sumolib.net.readNet(net_path)

with open('../Simulation_scenarios/motorway/Results/incident_large/ind_to_edge.json') as f:
    ind_to_edge = json.load(f)

*** DATA SUMMARY: ***
input_obs_full.shape=torch.Size([10270, 147, 6, 15, 3])
input_time_full.shape=torch.Size([10270, 147, 6, 15, 2])
target_full.shape=torch.Size([10270, 147, 4])
incident_info_full.shape=torch.Size([10270, 4])
network_info_full.shape=torch.Size([10270, 147, 13])



In [18]:
test_edges = ['4414080#0.187', 
                '4414080#0.756', 
                '4414080#0', 
                '332655581', 
                '360361373-AddedOffRampEdge', 
                '360361373.2643' ]

In [19]:
edge_to_ind = {v:k for k, v in ind_to_edge.items()}

In [28]:
test_edge_idxs = torch.tensor([int(edge_to_ind[edge]) for edge in test_edges])

In [29]:
test_edge_idxs

tensor([80, 81, 79, 53, 59, 64])

In [45]:
input_full = torch.Tensor(np.load(f'{folder_path}/input_data.npy'))
input_obs_full = input_full[...,:-2]
input_time_full = input_full[...,-2:]
target_full = torch.Tensor(np.load(f'{folder_path}/target_data.npy'))
incident_info_full = torch.Tensor(np.load(f'{folder_path}/incident_info.npy'))
network_info_full = torch.Tensor(np.load(f'{folder_path}/network_info.npy'))

In [74]:
spatial_test_mask = sum(incident_info_full[...,0] == edge_idx for edge_idx in test_edge_idxs).bool()

In [75]:
spatial_test_mask

tensor([False, False, False,  ..., False, False, False])

In [78]:
incident_info_full[spatial_test_mask]

tensor([[8.1000e+01, 4.0000e+00, 1.3800e+01, 1.0030e+03],
        [5.3000e+01, 1.0000e+00, 1.3800e+01, 1.2010e+03],
        [5.3000e+01, 1.0000e+00, 1.3800e+01, 1.2120e+03],
        ...,
        [8.0000e+01, 2.0000e+00, 1.3800e+01, 1.2180e+03],
        [7.9000e+01, 2.0000e+00, 1.3800e+01, 1.1860e+03],
        [5.3000e+01, 1.0000e+00, 1.3800e+01, 1.1570e+03]])